<a href="https://colab.research.google.com/github/namwootree/Basic_Skill/blob/main/%5BBaseline%5D_KFold(5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting

In [1]:
!nvidia-smi

Tue Oct  4 10:07:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

import lightgbm as lgb

import gc

# Load Data

In [17]:
df_submission_path = '/content/drive/MyDrive/머신러닝 엔지니어링/데이콘/제주도 도로 교통량 예측/data/sample_submission.csv'
df_train_path = '/content/drive/MyDrive/머신러닝 엔지니어링/데이콘/제주도 도로 교통량 예측/data/train.csv'
df_test_path = '/content/drive/MyDrive/머신러닝 엔지니어링/데이콘/제주도 도로 교통량 예측/data/test.csv'

In [18]:
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet(f'./{save_name}.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')

In [19]:
csv_to_parquet(df_train_path, 'train')
csv_to_parquet(df_test_path, 'test')

train Done.
test Done.


In [41]:
train = pd.read_parquet('/content/train.parquet')
test = pd.read_parquet('/content/test.parquet')

# Preprocessing

In [42]:
str_col = ['day_of_week','start_turn_restricted','end_turn_restricted']

for i in str_col:

    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):

        if label not in le.classes_: 
          
            le.classes_ = np.append(le.classes_, label)

    test[i]=le.transform(test[i])

In [43]:
y = train['target'] 

X = train.drop(['id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)
X = np.array(X.iloc[:,:-1])
test = test.drop(['id','base_date', 'road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)
test = np.array(test.iloc[:,:-1])

print(X_train.shape)
print(y_train.shape)
print(test.shape)

(2407024, 16)
(2407024,)
(291241, 16)


# Modeling

In [31]:
LR = lgb.LGBMRegressor(random_state=2022)

In [32]:
kf = KFold(n_splits=5, shuffle=True, random_state=2022)

In [44]:
num_model = 1

list_result = []

for train_index, text_index in kf.split(X_train):

  X_train, X_test = X[train_index], X[text_index]
  y_train, y_test = y[train_index], y[text_index]

  LR = lgb.LGBMRegressor(random_state=2022)

  LR.fit(X_train, y_train)

  y_pred = LR.predict(X_test)

  MSE = mean_squared_error(y_pred, y_test)

  print(f'{num_model} 번 Model Vaild MSE : {MSE}')

  num_model += 1

  result_pred = LR.predict(test)

  list_result.append(result_pred)

1 번 Model Vaild MSE : 40.414347646534026
2 번 Model Vaild MSE : 40.403430641966004
3 번 Model Vaild MSE : 40.716275788607945
4 번 Model Vaild MSE : 40.43471069922865
5 번 Model Vaild MSE : 40.30628597780492


In [46]:
result_1 = list_result[0]
result_2 = list_result[1]
result_3 = list_result[2]
result_4 = list_result[3]
result_5 = list_result[4]

list_final = []

for i in range(len(result_1)):

  final_result = (result_1[i] + result_2[i] + result_3[i] + result_4[i] + result_5[i]) / 5

  list_final.append(final_result)

# Submission

In [48]:
sample_submission = pd.read_csv(df_submission_path)

In [49]:
sample_submission['target'] = list_final
sample_submission.to_csv("./baseline_KFold.csv", index = False)

In [50]:
sample_submission

,id,target
0,TEST_000000,27.225040
1,TEST_000001,44.512729
2,TEST_000002,59.922350
3,TEST_000003,36.061080
4,TEST_000004,37.365421
...,...,...
291236,TEST_291236,47.924126
291237,TEST_291237,52.494172
291238,TEST_291238,21.415338
291239,TEST_291239,25.795687
